In [ ]:
import os
import pickle
import json
import shutil
import tensorflow as tf
import tensorflowjs as tfjs
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_curve, auc, classification_report, multilabel_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay
from tensorboard.plugins.hparams import api as hp
from imutils import paths
from ast import literal_eval

In [ ]:
run_env = 'local' # local, colab
ds_type = 'raw_images' # np_tensor_array, raw_images
datagen_api = 'keras' # keras, tf
model_to_load = 'last' # last, best_val_loss

method_name = 'EfficientNet-b7' # Model 1, ResNet50, EfficientNet-b7

train_the_model = True
predict_val_set = True
predict_test_set = True
save_tfjs_model = False

train_size=0.7
training_epochs = 20

In [ ]:
prediction_threshold = 0.7
target_size = (224,224)
batch_size = 64
random_state = 42

In [ ]:
if run_env == 'colab':
    from google.colab import drive
    drive.mount('/content/drive')
    working_dir = '/content/drive/MyDrive/PFE'
    input_dir = os.path.join(working_dir,'dataset')
    !cp {working_dir}/scripts . -r
else:
    working_dir = os.getcwd()
    input_dir = os.path.join(working_dir,'dataset')

from scripts.ProgressBar import ProgressBar
from scripts.TFImageDataGenerator import TFImageDataGenerator

In [ ]:
# Detect hardware, return appropriate distribution strategy

run_acc = 'cpu'

device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    run_acc = 'gpu'

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    run_acc = 'tpu'
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

In [ ]:
if not os.path.exists( working_dir+'/plots' ) : os.mkdir( working_dir + '/plots' )
plots_dir = os.path.join( working_dir, 'plots', method_name )
if not os.path.exists( plots_dir ) : os.mkdir( plots_dir )

if not os.path.exists( working_dir+'/saved_actions' ) : os.mkdir( working_dir + '/saved_actions' )
saved_actions_dir = os.path.join( working_dir, 'saved_actions', method_name )
if not os.path.exists( saved_actions_dir ) :  os.mkdir( saved_actions_dir )

tensorboard_callbacks_dir = os.path.join( saved_actions_dir, 'tensorboard_callbacks' )
if not os.path.exists( tensorboard_callbacks_dir ) :  os.mkdir( tensorboard_callbacks_dir )

pretrained_weights_dir = os.path.join( working_dir, 'pretrained_weights' )
if not os.path.exists( pretrained_weights_dir ) :  os.mkdir( pretrained_weights_dir )

In [ ]:
if ds_type == 'raw_images':
    train_set_input_file = pd.read_csv(input_dir+'/train_set.csv',converters={"label_names": literal_eval})
    _train_set_rel_paths = train_set_input_file['rel_paths']
    _train_set_label_names = train_set_input_file['label_names']
    train_set_dir = os.path.join( input_dir, 'train_set' )
else:
    train_set_input_object = np.load(input_dir+'/train_set.npz',allow_pickle=True)
    train_set_image_tensors = train_set_input_object['image_tensors']
    train_set_label_names = train_set_input_object['label_names']

test_set_input_file = pd.read_csv(input_dir+'/test_set.csv',converters={"label_names": literal_eval})
test_set_rel_paths = test_set_input_file['rel_paths']
test_set_label_names = test_set_input_file['label_names']

test_set_dir = os.path.join( input_dir, 'test_set' )

In [ ]:
if ds_type == 'raw_images':
    (train_set_rel_paths, val_set_rel_paths, train_set_label_names, val_set_label_names, train_set_indices, val_set_indices) = train_test_split( _train_set_rel_paths, _train_set_label_names, np.arange(len(_train_set_rel_paths)), train_size=train_size, random_state=random_state)
else:
    (train_set_image_tensor_ids, val_set_image_tensor_ids, train_set_label_names, val_set_label_names) = train_test_split( np.arange( len(train_set_image_tensors) ), train_set_label_names, train_size=train_size, random_state=random_state)
    val_set_image_tensors = train_set_image_tensors[val_set_image_tensor_ids]
    train_set_image_tensors = train_set_image_tensors[train_set_image_tensor_ids]

train_set_label_binarizer = MultiLabelBinarizer()
val_set_label_binarizer = MultiLabelBinarizer()
test_set_label_binarizer = MultiLabelBinarizer()

train_set_labels = train_set_label_binarizer.fit_transform(train_set_label_names)
val_set_labels = val_set_label_binarizer.fit_transform(val_set_label_names)
test_set_labels = test_set_label_binarizer.fit_transform(test_set_label_names)

dataset_classes = train_set_label_binarizer.classes_
testset_classes = test_set_label_binarizer.classes_

In [ ]:
train_set_data_class_count = [0] * (len(dataset_classes)+1)
val_set_data_class_count = [0] * (len(dataset_classes)+1)
test_set_data_class_count = [0] * (len(dataset_classes)+1)
total_dataset_data_class_count = [0] * (len(dataset_classes)+1)

for train_set_data_index in range( len(train_set_labels) ):
    train_set_data_class_count[len(dataset_classes)] += 1
    total_dataset_data_class_count[len(dataset_classes)] += 1
    for train_set_label_index in range( len(dataset_classes) ) :
        if ( train_set_labels[train_set_data_index][train_set_label_index] == 1 ):          
            train_set_data_class_count[train_set_label_index] += 1
            total_dataset_data_class_count[train_set_label_index] += 1         

for val_set_data_index in range( len(val_set_labels) ):
    val_set_data_class_count[len(dataset_classes)] += 1
    total_dataset_data_class_count[len(dataset_classes)] += 1
    for val_set_label_index in range( len(dataset_classes) ) :
        if ( val_set_labels[val_set_data_index][val_set_label_index] == 1 ):          
            val_set_data_class_count[val_set_label_index] += 1
            total_dataset_data_class_count[val_set_label_index] += 1

for test_set_data_index in range( len(test_set_labels) ):
    test_set_data_class_count[len(dataset_classes)] += 1
    total_dataset_data_class_count[len(dataset_classes)] += 1
    for test_set_label_index in range( len(dataset_classes) ) :
        for test_set_label_test_index in range( len(testset_classes) ) :
            if ( test_set_labels[test_set_data_index][test_set_label_test_index] == 1 and dataset_classes[test_set_label_index] == testset_classes[test_set_label_test_index] ):          
                test_set_data_class_count[test_set_label_index] += 1
                total_dataset_data_class_count[test_set_label_index] += 1

fig, ax = plt.subplots(1,1)
data = [
    train_set_data_class_count,
    val_set_data_class_count,
    test_set_data_class_count,
    total_dataset_data_class_count
]
row_labels = ['Training images','Validation images', 'Test images', 'Total Images']
column_labels = dataset_classes.tolist() + [ 'All Classes' ]

df=pd.DataFrame(data,columns=column_labels)
ax.set_title('Wheat Disease Identification Dataset Summary',fontweight ='bold')
ax.axis('off')
dataset_summary_table = ax.table(cellText=df.values,colLabels=df.columns,rowLabels=row_labels,loc='center')
dataset_summary_table.scale(3, 3)
plt.savefig(plots_dir+'/dataset_summary.png',bbox_inches='tight')
plt.show()
plt.clf()

In [ ]:
if ds_type == 'raw_images':
    train_set_df = {
        'abs_paths': [],
        'label_names': []
    }
    for train_set_index in train_set_indices:
        train_set_df['abs_paths'].append(os.path.abspath(train_set_dir+train_set_rel_paths[train_set_index]))
        train_set_df['label_names'].append(train_set_label_names[train_set_index])

    train_set_df = pd.DataFrame(train_set_df)

    val_set_df = {
        'abs_paths': [],
        'label_names': []
    }
    for val_set_index in val_set_indices:
        val_set_df['abs_paths'].append(os.path.abspath(train_set_dir+val_set_rel_paths[val_set_index]))
        val_set_df['label_names'].append(val_set_label_names[val_set_index])

    val_set_df = pd.DataFrame(val_set_df)

test_set_df = {
    'abs_paths': [],
    'label_names': []
}
for test_set_index in range(len(test_set_labels)):
    test_set_df['abs_paths'].append(os.path.abspath(test_set_dir+test_set_rel_paths[test_set_index]))
    test_set_df['label_names'].append(test_set_label_names[test_set_index])

test_set_df = pd.DataFrame(test_set_df)
    
val_set_labels = test_set_labels
val_set_label_names = test_set_label_names
val_set_rel_paths = test_set_rel_paths
val_set_df = test_set_df

In [ ]:
preprocessing_function = tf.keras.applications.vgg16.preprocess_input

if method_name == 'ResNet50':
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input

if method_name == 'ResNet50V2':
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input

elif method_name == 'InceptionV3':
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input

elif method_name.split('-')[0] == 'EfficientNet':
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input # no longer does anything
    
elif method_name == 'MobileNet':
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input

elif method_name == 'MobileNetV2':
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    
elif method_name == 'VGG16':
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
    
elif method_name == 'VGG19':
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input

In [ ]:
if datagen_api == 'keras':
    ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
else:
    ImageDataGenerator = TFImageDataGenerator
    
augmented_datagen = ImageDataGenerator(
    preprocessing_function=preprocessing_function,
    rotation_range=30,
    shear_range=15,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8,1.2]
)

flat_datagen = ImageDataGenerator(
    preprocessing_function=preprocessing_function,
)

if ds_type == 'raw_images':
    train_set_datagen = augmented_datagen.flow_from_dataframe(
        dataframe=train_set_df,
        target_size=target_size,
        x_col="abs_paths",
        y_col="label_names",
        batch_size=batch_size,
        interpolation='lanczos',
    )
    val_set_datagen = flat_datagen.flow_from_dataframe(
        dataframe=val_set_df,
        target_size=target_size,
        x_col="abs_paths",
        y_col="label_names",
        batch_size=batch_size,
        interpolation='lanczos',
        shuffle=False
    )
else:
    train_set_datagen = augmented_datagen.flow(train_set_image_tensors, train_set_labels, batch_size=batch_size)
    val_set_datagen = flat_datagen.flow(val_set_image_tensors, val_set_labels, batch_size=batch_size, shuffle=False)

In [ ]:
if method_name == 'Model 1':
    with strategy.scope():
        # Sequential Model
        model = tf.keras.models.Sequential()
        # 1st layer
        model.add(tf.keras.layers.Conv2D(16,(3,3),input_shape = (*target_size,3),padding='same',activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2,2))
        # 2nd Layer
        model.add(tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2,2))
        # Flatten Layer
        model.add(tf.keras.layers.Flatten())
        # 1st Hidden Layer
        model.add(tf.keras.layers.Dense(512,activation='relu',))
        model.add(tf.keras.layers.Dropout(0.4))
        # 2nd Hidden Layer
        model.add(tf.keras.layers.Dense(256,activation='relu'))
        # Output Layer
        model.add(tf.keras.layers.Dense(len(dataset_classes),activation='sigmoid'))

        opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
        model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
if method_name == 'ResNet50':
    with strategy.scope():
        base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=tf.keras.layers.Input(shape=(*target_size, 3)))
        # num_layers = 175
        # setting all except last 44 layers as untrainable
        for layer in base_model.layers[:-44]:
            layer.trainable = False
        
        #taking all except last 15 layers
        model_outputs = base_model.layers[-15].output
        model_outputs = tf.keras.layers.Flatten()(model_outputs)
        model_outputs = tf.keras.layers.Dropout(0.8)(model_outputs)
        model_outputs = tf.keras.layers.Dense(len(dataset_classes), activation='sigmoid')(model_outputs)

        model = tf.keras.models.Model(inputs=base_model.input, outputs=model_outputs)

        opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
        model.compile( loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
if method_name.split('-')[0] == 'EfficientNet':
    version = method_name.split('-')[1]
    if not os.path.exists( pretrained_weights_dir+'/noisy-student-'+version+'.h5' ):
        !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/noisystudent/noisy_student_efficientnet-{version}.tar.gz
        !tar -xf noisy_student_efficientnet-{version}.tar.gz
        os.remove('noisy_student_efficientnet-'+version+'.tar.gz')
        if not os.path.exists( 'scripts/efficientnet_weight_update_util.py' ):
            !wget https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/python/keras/applications/efficientnet_weight_update_util.py -O {working_dir}/scripts/efficientnet_weight_update_util.py
            if run_env == 'colab':
                !cp {working_dir}/scripts . -r
        if version == 'b0':
            !python scripts/efficientnet_weight_update_util.py --model b0 --notop --ckpt \
                noisy_student_efficientnet-b0/model.ckpt --o {pretrained_weights_dir}/noisy-student-b0.h5
            shutil.rmtree('noisy_student_efficientnet-b0')
        else:
            !python scripts/efficientnet_weight_update_util.py --model {version} --notop --ckpt \
                noisy-student-efficientnet-{version}/model.ckpt --o {pretrained_weights_dir}/noisy-student-{version}.h5
            shutil.rmtree('noisy-student-efficientnet-'+version)

    with strategy.scope():
        if version == 'b0':
            bm = tf.keras.applications.EfficientNetB0
        elif version == 'b1':
            bm = tf.keras.applications.EfficientNetB1
        elif version == 'b2':
            bm = tf.keras.applications.EfficientNetB2
        elif version == 'b3':
            bm = tf.keras.applications.EfficientNetB3
        elif version == 'b4':
            bm = tf.keras.applications.EfficientNetB4
        elif version == 'b5':
            bm = tf.keras.applications.EfficientNetB5
        elif version == 'b6':
            bm = tf.keras.applications.EfficientNetB6
        elif version == 'b7':
            bm = tf.keras.applications.EfficientNetB7
    
        base_model = bm( weights=pretrained_weights_dir+'/noisy-student-'+version+'.h5', include_top=False ,input_shape=(*target_size,3))

        base_model.trainable = True

        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(len(dataset_classes), activation='sigmoid')
        ])

        opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
        model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()
tf.keras.utils.plot_model( model, show_shapes=True, show_layer_names=True, to_file=plots_dir+'/model.png')

In [ ]:
# Learning rate callback
def learning_rate_adjust(epoch):
    LR_START = 0.00001
    LR_MAX = 0.00005 * strategy.num_replicas_in_sync
    LR_MIN = 0.00001
    LR_RAMPUP_EPOCHS = 5
    LR_SUSTAIN_EPOCHS = 0
    LR_EXP_DECAY = .8
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
learning_rate_callback = tf.keras.callbacks.LearningRateScheduler(learning_rate_adjust, verbose=1)

if method_name.split('-')[0] == 'EfficientNet':
    rang = np.arange(training_epochs)
    y = [learning_rate_adjust(x) for x in rang]
    x_bounds = np.arange(1, training_epochs+1)
    x_lim = [1, training_epochs]
    if (training_epochs<=20): plt.xticks(np.arange(round(0,training_epochs), training_epochs, 2))
    plt.plot(x_bounds, y)
    plt.xlim(x_lim)
    plt.ylabel('Learning rate')
    plt.xlabel('Epoch')
    plt.title("Learning rate per epoch")
    plt.grid(True)
    plt.savefig(plots_dir+'/learning_rate_per_epoch.png', dpi=140)
    plt.show()
    plt.clf()

# tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(tensorboard_callbacks_dir, histogram_freq=1)

# checkpoint callback
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= saved_actions_dir+'/model_best_val_loss',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

# early stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
csv_logger_callback = tf.keras.callbacks.CSVLogger(saved_actions_dir+'/model_log.csv', append=False, separator=',')

callbacks = []
if method_name.split('-')[0] == 'EfficientNet':
    callbacks.append(learning_rate_callback)
if not run_acc == 'tpu':
    callbacks.append(tensorboard_callback)
    callbacks.append(model_checkpoint_callback)
callbacks.append(early_stopping_callback)
callbacks.append(csv_logger_callback)

In [ ]:
if train_the_model:
    
    def warn_bypass(*args, **kwargs):
            pass
    import warnings
    warn = warnings.warn
    warnings.warn = warn_bypass

    eargs = {}
    if ds_type == 'raw_images':
        eargs['workers'] = 8

    try:
        model_fit_object = model.fit(
            train_set_datagen,
            validation_data=val_set_datagen,
            validation_steps=len(val_set_labels) // batch_size,
            steps_per_epoch=len(train_set_labels) // batch_size,
            epochs=training_epochs,
            callbacks = callbacks,
            **eargs
        )
        
        model_history = model_fit_object.history
        np.save(saved_actions_dir+'/model_history.npy', model_history)
    except KeyboardInterrupt:
        pass
    
    model.save(saved_actions_dir+'/model.h5')
    
    warnings.warn = warn

In [ ]:
model_history = np.load(saved_actions_dir+'/model_history.npy',allow_pickle='TRUE').item()

##################

bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
arrowprops=dict(arrowstyle="-",connectionstyle="angle,angleA=0,angleB=120")
kw = dict(xycoords='data',textcoords="axes fraction", arrowprops=arrowprops, bbox=bbox_props, ha="right", va="bottom")

max_accuracy = max(model_history['accuracy'])
max_accuracy_epoch = model_history['accuracy'].index(max_accuracy) + 1
max_accuracy_text= "max={:.4f} at epoch {:.0f}".format(max_accuracy, max_accuracy_epoch)

max_val_accuracy = max(model_history['val_accuracy'])
max_val_accuracy_epoch = model_history['val_accuracy'].index(max_val_accuracy) + 1
max_val_accuracy_text= "max={:.4f} at epoch {:.0f}".format(max_val_accuracy, max_val_accuracy_epoch)

min_loss = min(model_history['loss'])
min_loss_epoch = model_history['loss'].index(min_loss) + 1
min_los_text= "min={:.4f} at epoch {:.0f}".format(min_loss, min_loss_epoch)

min_val_loss = min(model_history['val_loss'])
min_val_loss_epoch = model_history['val_loss'].index(min_val_loss) + 1
min_val_loss_text= "min={:.4f} at epoch {:.0f}".format(min_val_loss, min_val_loss_epoch)

##################

N = len(model_history['accuracy'])
x_bounds = np.arange(1, N+1)
x_lim = [1, N]

##################

plt.figure(figsize=(12, 9))
plt.plot(x_bounds, model_history['accuracy'], label='Training Accuracy')
plt.plot(x_bounds, model_history['val_accuracy'], label='Validation Accuracy')
plt.title(method_name+' Wheat Disease Identification Model Training vs Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
y=model_history['accuracy'] + model_history['val_accuracy']
plt.yticks(np.arange(round(min(y),1), 1., 0.05))
if N<=30: plt.xticks(np.arange(round(0,N), N, 2))
plt.xlim(x_lim)
plt.grid(True)
plt.legend(loc='lower right')
plt.annotate(max_accuracy_text, xy=(max_accuracy_epoch, max_accuracy), xytext=(0.7,max_accuracy+0.03), **kw)
plt.annotate(max_val_accuracy_text, xy=(max_val_accuracy_epoch, max_val_accuracy), xytext=(0.95,max_val_accuracy-0.07), **kw)
plt.savefig(plots_dir+'/accuracy.png', dpi=140)
#plt.show()
plt.clf()

##################

plt.figure(figsize=(12, 9))
plt.plot(x_bounds,model_history['loss'], label='Training Loss')
plt.plot(x_bounds,model_history['val_loss'], label='Validation Loss')
plt.title(method_name+' Wheat Disease Identification Model Training vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
y=model_history['loss'] + model_history['val_loss']
factor = np.ceil(max(y))
plt.yticks(np.arange(round(min(y),1), round(max(y),1)+0.1*factor, 0.05*factor))
if N<=30: plt.xticks(np.arange(round(0,N), N, 2))
plt.xlim(x_lim)
plt.grid(True)
plt.legend(loc='upper right')
plt.annotate(min_los_text, xy=(min_loss_epoch, min_loss), xytext=(0.95,min_loss+0.1), **kw)
plt.annotate(min_val_loss_text, xy=(min_val_loss_epoch, min_val_loss), xytext=(0.7,min_val_loss+0.1), **kw)
plt.savefig(plots_dir+'/loss.png', dpi=140)
#plt.show()
plt.clf()

##################

plt.figure(figsize=(25, 10))

##################

ax = plt.subplot(1, 2, 1)
ax.plot(x_bounds, model_history['accuracy'], label='Training Accuracy')
ax.plot(x_bounds, model_history['val_accuracy'], label='Validation Accuracy')
ax.set_title(method_name+' Wheat Disease Identification Model Training vs Validation Accuracy')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epoch')
y=model_history['accuracy'] + model_history['val_accuracy']
ax.set_yticks(np.arange(round(min(y),1), 1., 0.05))
if N<=30: ax.set_xticks(np.arange(round(0,N), N, 2))
ax.set_xlim(x_lim)
ax.grid(True)
ax.legend(loc='lower right')
ax.annotate(max_accuracy_text, xy=(max_accuracy_epoch, max_accuracy), xytext=(0.7,max_accuracy+0.03), **kw)
ax.annotate(max_val_accuracy_text, xy=(max_val_accuracy_epoch, max_val_accuracy), xytext=(0.95,max_val_accuracy-0.07), **kw)

##################

ax2 = plt.subplot(1, 2, 2)
ax2.plot(x_bounds, model_history['loss'], label='Training Loss')
ax2.plot(x_bounds, model_history['val_loss'], label='Validation Loss')
ax2.set_title(method_name+' Wheat Disease Identification Model Training vs Validation Loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
y=model_history['loss'] + model_history['val_loss']
factor = np.ceil(max(y))
ax2.set_yticks(np.arange(round(min(y),1), round(max(y),1)+0.1*factor, 0.05*factor))
if N<=30: ax2.set_xticks(np.arange(round(0,N), N, 2))
ax2.set_xlim(x_lim)
ax2.grid(True)
ax2.legend(loc='upper right')
ax2.annotate(min_los_text, xy=(min_loss_epoch, min_loss), xytext=(0.95,min_loss+0.1), **kw)
ax2.annotate(min_val_loss_text, xy=(min_val_loss_epoch, min_val_loss), xytext=(0.7,min_val_loss+0.1), **kw)

##################

#plt.savefig(plots_dir+'/accuracy_loss.png')
plt.show()
plt.clf()

In [ ]:
if save_tfjs_model or predict_val_set or predict_test_set:
    if not model_to_load == 'last':
        model = tf.keras.models.load_model(saved_actions_dir+'/model_'+model_to_load)
    else:
        model = tf.keras.models.load_model(saved_actions_dir+'/model.h5')

In [ ]:
if save_tfjs_model:
    tfjs.converters.save_keras_model(model, saved_actions_dir+'/model_tfjs')
    with open(saved_actions_dir+'/model_tfjs/dataset_classes.json', "w") as json_file:
        json.dump(dataset_classes.tolist(), json_file, ensure_ascii=False)

In [ ]:
if predict_val_set:
    val_set_pred_labels = model.predict(val_set_datagen, batch_size=batch_size)
    np.save( saved_actions_dir+'/val_set_pred_labels', val_set_pred_labels )

In [ ]:
val_set_pred_labels = np.load(saved_actions_dir+'/val_set_pred_labels.npy')

val_set_pred_labels_normalized = []

for prediction in val_set_pred_labels:
    to_append = [1 if i>=prediction_threshold else 0 for i in prediction]
    if not any(to_append):
        new_threshold = max(prediction) - 0.1
        if new_threshold > 0.1:
            to_append = [1 if i>=new_threshold else 0 for i in prediction]
        else:
            to_append = [1 if i==max(prediction) else 0 for i in prediction]
    val_set_pred_labels_normalized.append(to_append)
val_set_pred_labels_normalized = np.array(val_set_pred_labels_normalized)

In [ ]:
cr = classification_report(val_set_labels, val_set_pred_labels_normalized, target_names=dataset_classes, output_dict=True)
plt.figure(figsize=(15, 7))
ax = sns.heatmap(pd.DataFrame(cr).T, annot=True, annot_kws={"size": 12}, cmap=ListedColormap(['white']), linecolor='#efefef', linewidths=2, cbar=False, fmt='.4g')
plt.title(method_name+' Classification Report',fontweight ='bold')
plt.savefig(plots_dir+'/classification_report.png', dpi=140)
plt.show()
plt.clf()
sns.reset_orig()

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(dataset_classes)):
    fpr[i], tpr[i], _ = roc_curve(val_set_labels[:, i], val_set_pred_labels[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(val_set_labels.ravel(), val_set_pred_labels.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(dataset_classes))]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(len(dataset_classes)):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= len(dataset_classes)

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(12, 9))
plt.plot(fpr["micro"], tpr["micro"], label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]), color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"], label='macro-average ROC curve (area = {0:0.2f})'.format(roc_auc["macro"]), color='navy', linestyle=':', linewidth=4)

for i in range(len(dataset_classes)):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class "{0}" (area = {1:0.2f})'.format(dataset_classes[i], roc_auc[i]))

plt.plot([0, 1], [0, 1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.yticks(np.arange(0.1, 1.05, 0.1))
plt.xticks(np.arange(0.1, 1., 0.1))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class: '+method_name,fontweight ='bold')
plt.legend(loc="lower right")
plt.savefig(plots_dir+'/roc_curves.png', dpi=140)
plt.show()
plt.clf()

In [ ]:
# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(len(dataset_classes)):
    precision[i], recall[i], _ = precision_recall_curve(val_set_labels[:, i], val_set_pred_labels[:, i])
    average_precision[i] = average_precision_score(val_set_labels[:, i], val_set_pred_labels[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(val_set_labels.ravel(),
    val_set_pred_labels.ravel())
average_precision["micro"] = average_precision_score(val_set_labels, val_set_pred_labels, average="micro")

plt.figure(figsize=(12, 7))
plt.step(recall['micro'], precision['micro'], where='post')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid(True)
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(method_name+' Average precision score, micro-averaged over all classes: AP={0:0.4f}'.format(average_precision["micro"]))

plt.savefig(plots_dir+'/average-precision-score.png', dpi=140)
plt.show()
plt.clf()

In [ ]:
plt.figure(figsize=(16, 12))
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

lines.append(l)
labels.append('iso-f1 curves')
l, = plt.plot(recall["micro"], precision["micro"], color='gold', linestyle=':')
lines.append(l)
labels.append('micro-average Precision-recall (area = {0:0.2f})'.format(average_precision["micro"]))

for i in range(len(dataset_classes)):
    l, = plt.plot(recall[i], precision[i])
    lines.append(l)
    labels.append('Precision-recall for class "{0}" (area = {1:0.2f})'.format(dataset_classes[i], average_precision[i]))

fig = plt.gcf()
fig.subplots_adjust(bottom=0.3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(method_name+' Extension of Precision-Recall curve to multi-class')
plt.legend(lines, labels, loc=(0, -.5), prop=dict(size=14))


plt.savefig(plots_dir+'/precision-recall_curves.png', dpi=140)
plt.show()
plt.clf()

In [ ]:
cm = multilabel_confusion_matrix(val_set_labels, val_set_pred_labels_normalized)
fig, axes = plt.subplots( round(len(cm)/2), 2, figsize=(20, 35))
axes = axes.ravel()

plt.title(method_name+' Wheat Disease Identification Confusion Matrix')
for cm_index in range(len(cm)):
    cmp = ConfusionMatrixDisplay(cm[cm_index],display_labels=['0','1'])
    ax = plt.subplot( round( len(cm) / 2 ) , 2 , cm_index + 1 )
    ax.set_title(dataset_classes[cm_index])
    cmp.plot(ax=ax)

plt.savefig(plots_dir+'/confusion_matrix.png', dpi=200)
plt.show()
plt.clf()

In [ ]:
val_set_pred_labels_max_indices = []
val_set_labels_max_indices = []
for item_index in range(len(val_set_labels)):
    true_item = val_set_labels[item_index]
    pred_item = val_set_pred_labels[item_index]
    pred_item_normalized = val_set_pred_labels_normalized[item_index]
    
    pred_max_val = max(pred_item)
    pred_max_index = pred_item.argmax()
    
    for i in range(len(dataset_classes)):
        pred_index = pred_item.argmax()
        if true_item[pred_index] == 1 and pred_item_normalized[pred_index] == 1:
            val_set_pred_labels_max_indices.append(pred_index)
            val_set_labels_max_indices.append(pred_index)
            break
        else:
            if i == len(dataset_classes)-1:
                val_set_pred_labels_max_indices.append(pred_max_index)
                val_set_labels_max_indices.append(true_item.argmax())
            else:
                pred_item = np.delete(pred_item, pred_index)
                
cm = confusion_matrix(val_set_labels_max_indices, val_set_pred_labels_max_indices)
cmp = ConfusionMatrixDisplay(cm, display_labels=dataset_classes)
fig, ax = plt.subplots(figsize=(20,15))
ax.set_title(method_name+' Wheat Disease Identification Confusion Matrix',fontweight ='bold')
cmp.plot(ax=ax)
    
plt.savefig(plots_dir+'/confusion_matrix.png')
plt.show()
plt.clf()

In [ ]:
if predict_test_set:
    plt.subplots(figsize=(15, len(test_set_rel_paths)*3))

    progress = ProgressBar( len(test_set_rel_paths), fmt=ProgressBar.FULL, init="Predicting" )

    for test_image_index in range(len(test_set_rel_paths)):

        progress.current += 1
        progress()

        image = tf.keras.preprocessing.image.load_img( test_set_dir + test_set_rel_paths[test_image_index] )
        width, height = image.size
        image_resized = image.resize(target_size)
        img_array = tf.keras.preprocessing.image.img_to_array( image_resized )
        img_array = tf.expand_dims(img_array, 0)  # Create batch axis
        img_array = preprocessing_function(img_array)

        prediction = model.predict( img_array )

        result = ""
        for prediction_class_index in range( len(prediction[0]) ):
            result = result + dataset_classes[prediction_class_index] + ": " + str(round(100*prediction[0][prediction_class_index],2)) + "%\n"     

        ax = plt.subplot( round( len(test_set_rel_paths) / 2 ) , 2 , test_image_index + 1 )

        predicted = ""
        predicted_class_indices = [i for i, x in enumerate(prediction[0]) if x >= prediction_threshold]
        if len(predicted_class_indices) == 0:
            new_threshold = max(prediction[0]) - 0.1
            if new_threshold > 0.1:
                predicted_class_indices = [i for i, x in enumerate(prediction[0]) if x >= new_threshold]
            else:
                predicted_class_indices = [prediction[0].argmax()]
        cnt = 0
        for predicted_class_index in predicted_class_indices:
            predicted += dataset_classes[predicted_class_index]
            if cnt < len(predicted_class_indices)-1: predicted += ", "
            cnt += 1
        true = ""
        true_class_indices = [i for i, x in enumerate(test_set_labels[test_image_index]) if x == 1]
        cnt = 0
        for true_class_index in true_class_indices:
            true += testset_classes[true_class_index]
            if cnt < len(true_class_indices)-1: true += ", "
            cnt += 1

        ax.set_title( 'Predicted: ' + predicted )
        ax.set_yticks([ height / 2 ])
        ax.set_yticklabels([result])
        ax.set_xticks([])

        ax.set_xlabel( 'True: ' + true )

        ax.imshow( image, extent=[0,width,0,height] )

    progress.done()
    plt.show()
    plt.clf()